First, let's parse the instructions.

In [1]:
from enum import Enum
from typing import NamedTuple, List, Callable, Dict

In [2]:
class Operation(Enum):
    TURN_ON = "turn on"
    TURN_OFF = "turn off"
    TOGGLE = "toggle"

class Instruction(NamedTuple):
    op: Operation
    col_start: int
    row_start: int
    col_end: int
    row_end: int

In [3]:
def parse_line(line: str) -> Instruction:
    elements = line.split()
    
    end_coords = elements.pop().split(',')
    elements.pop()  # pop "through" string
    start_coords = elements.pop().split(',')
    op = " ".join(elements)
    
    return Instruction(
        Operation(op), 
        int(start_coords[0]),
        int(start_coords[1]),
        int(end_coords[0]),
        int(end_coords[1]),
    )

    
with open("day-6-input.txt") as file:
    instructions = [parse_line(line) for line in file.readlines()]

# "Normal" Solution

In [4]:
def find_solution(instructions: List[Instruction],
                  op_map: Dict[Operation, Callable[[int], int]],
                  count_func: Callable[[List[List[int]]], None]) -> None:
    
    num_cols = num_rows = 1000
    # 0 for light off, 1+ for light on.
    grid = [[0 for c in range(num_cols)] for r in range(num_rows)]
    
    for instruction in instructions:
        op_func = op_map[instruction.op]
        for r in range(instruction.row_start, instruction.row_end+1):
            for c in range(instruction.col_start, instruction.col_end+1):
                grid[r][c] = op_func(grid[r][c])

    count_func(grid)

## Part 1

In [5]:
op_map_1 = {
    Operation.TURN_ON: lambda state: 1,
    Operation.TURN_OFF: lambda state: 0,
    Operation.TOGGLE: lambda state: state ^ 1,
}

In [6]:
def count_lights(grid: List[List[int]]) -> None:
    count = 0
    for row in grid:
        for cell in row:
            if cell == 1:
                count += 1
    print("Total lights lit:", count)

In [7]:
find_solution(instructions, op_map_1, count_lights)

Total lights lit: 377891


## Part 2

In [8]:
op_map_2 = {
    Operation.TURN_ON: lambda state: state + 1,
    Operation.TURN_OFF: lambda state: max(state - 1, 0),
    Operation.TOGGLE: lambda state: state + 2,
}

In [9]:
def measure_brightness(grid: List[List[int]]) -> None:
    total = 0
    for row in grid:
        for cell in row:
            total += cell
    
    print("Total brightness:", total)

In [10]:
find_solution(instructions, op_map_2, measure_brightness)

Total brightness: 14110788


# Bit Manipulation Solution
Part 1 could be done with bit manipulation in Python, so I thought I'd code it up for fun. However, this feels pretty bad because I'm literally making numbers up to $2^{1,000,000}$.

In [11]:
class BitGrid:
    
    def __init__(self, num_cols: int, num_rows: int):
        self._num_cols = num_cols
        self._num_rows = num_rows
        self._value = 0
        
    def follow_instructions(self, instructions: List[Instruction]) -> None:
        for instruction in instructions:
            self._follow_instruction(instruction)
    
    def _follow_instruction(self, instruction: Instruction) -> None:
        if instruction.op == Operation.TURN_ON:
            self._turn_on(instruction)
        elif instruction.op == Operation.TURN_OFF:
            self._turn_off(instruction)
        elif instruction.op == Operation.TOGGLE:
            self._toggle(instruction)
        else:
            raise ValueError("Unexpected operation:", instruction.op)

    def _turn_on(self, instruction: Instruction) -> None:
        """Set bits."""
        mask_len = instruction.col_end - instruction.col_start + 1
        mask = 2**mask_len - 1
        for r in range(instruction.row_start, instruction.row_end+1):
            shift = r * self._num_cols + instruction.col_start
            self._value = self._value | (mask << shift)
            
    def _turn_off(self, instruction: Instruction) -> None:
        """Clear bits."""
        mask_len = instruction.col_end - instruction.col_start + 1
        mask = 2**mask_len - 1
        for r in range(instruction.row_start, instruction.row_end+1):
            shift = r * self._num_cols + instruction.col_start
            self._value = self._value & ~(mask << shift)
        
    def _toggle(self, instruction: Instruction) -> None:
        """Toggle bits with XOR."""
        mask_len = instruction.col_end - instruction.col_start + 1
        mask = 2**mask_len - 1
        for r in range(instruction.row_start, instruction.row_end+1):
            shift = r * self._num_cols + instruction.col_start
            self._value = self._value ^ (mask << shift)
            
    def count_lights(self) -> int:
        """Give up and and don't use bit manipulation due to performance."""
        return bin(self._value).count("1")

In [12]:
num_cols = num_rows = 1000
bit_grid = BitGrid(num_cols, num_rows)
bit_grid.follow_instructions(instructions)
bit_grid.count_lights()

377891